In [20]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd
import re

url = 'https://store.steampowered.com/search/?sort_by=Released_DESC&tags=-1&category1=998&page=1'

In [21]:
r = requests.get(url)

In [22]:
bs = BeautifulSoup(r.text, 'html.parser')

In [23]:
games = bs.findAll('div', 'responsive_search_name_combined')
print("Found {} games on the web page\n".format(len(games)))

Found 25 games on the web page



In [24]:
for game in games:
    title = game.find('span', 'title').contents[0]
    if not game.find('div', 'col search_released responsive_secondrow').contents:
        release_date = 'Unknown'
    else:
        release_date = game.find('div', 'col search_released responsive_secondrow').contents[0]
    if game.find('div', 'col search_discount responsive_secondrow').find('span') is not None:
        discount = game.find('div', 'col search_discount responsive_secondrow').find('span').contents[0]
        usual_price = game.find('div', 'col search_price discounted responsive_secondrow').find('span').find('strike').contents[0]
        discounted_price = game.find('div', 'col search_price discounted responsive_secondrow').contents[3]
    else:
        discount = None
        discounted_price = None
        usual_price = ' '.join(game.find('div', 'col search_price responsive_secondrow').contents[0].split())
    
    #print ('Title: {}\nRelease date: {}\nDiscount: {}\nUsual price: {}\nDiscounted price: {}\n'.format(title,release_date, discount, usual_price, discounted_price))

In [25]:
def get_game_data(game):
    item = {}
    title = game.find('span', 'title').contents[0]
    if not game.find('div', 'col search_released responsive_secondrow').contents:
        release_date = 'Unknown'
    else:
        release_date = game.find('div', 'col search_released responsive_secondrow').contents[0]
    if game.find('div', 'col search_discount responsive_secondrow').find('span') is not None:
        discount = game.find('div', 'col search_discount responsive_secondrow').find('span').contents[0]
        usual_price = game.find('div', 'col search_price discounted responsive_secondrow').find('span').find('strike').contents[0]
        discounted_price = ' '.join(game.find('div', 'col search_price discounted responsive_secondrow').contents[3].split())
    else:
        discount = None
        discounted_price = None
        usual_price = ' '.join(game.find('div', 'col search_price responsive_secondrow').contents[0].split())
    item = {
        'title': title,
        'release_date': release_date,
        'discount': discount,
        'usual_price': usual_price,
        'discounted_price': discounted_price
    }
    return item

def crawl_page(startswith):
    url = 'https://store.steampowered.com/search/?sort_by=Released_DESC&tags=-1&category1=998&page={}'.format(startswith)
    r = requests.get(url)
    bs = BeautifulSoup(r.text, 'html.parser')
    data, item = [], {}
    games = bs.findAll('div', 'responsive_search_name_combined')
    for game in games:
        item = get_game_data(game)
        data.append(item)
    return data

data = crawl_page(0)
df = pd.DataFrame(data)
print(df)

                                                title        release_date  \
0                                          Astro Dash         5 Dec, 2020   
1                                         Last of Ass         5 Dec, 2020   
2                                             Puzzzle         5 Dec, 2020   
3                                      Primate Signal         5 Dec, 2020   
4                                 Public Defense Corp         4 Dec, 2020   
5                                       Hanako | 花子さん             Unknown   
6                                            OPERATOR         COMING SOON   
7                                     Summon & Fusion         4 Dec, 2020   
8                               A Monster, The Mauler         4 Dec, 2020   
9                              ZERONE - Spinoff Arena            Dec 2020   
10                                        Tank Firing         4 Dec, 2020   
11                        Snowball Fight At Christmas         4 Dec, 2020   

In [26]:
nb_games_per_page = 25
nb_games = bs.find('div', 'responsive_page_template_content').find('script').contents[0]
nItemCount = nb_games.find('nItemCount')
nb_games = nb_games[nItemCount:]
nb_games = nb_games[:nb_games.index(';')]
nb_games = int(nb_games.split()[-1])
nb_pages = int(np.ceil(nb_games / nb_games_per_page))
print(nb_pages)

1675
